# Numerical Integration

In [10]:
function trapezoid(f, a, b, n)
    h = (b-a) / n
    domain = a:h:b
    
    pairs = zip(domain[1:end-1], domain[2:end])
    sum(h * (f(an) + f(bn)) / 2 for (an, bn) in pairs)
end

trapezoid (generic function with 2 methods)

In [11]:
trapezoid(x -> exp(-x^2), 0, 1, 1000)

0.7468240714991848